In [1]:
import pandas as pd
import numpy as np
#%pip install faker
from faker import Faker
from scipy.stats import norm
import calendar
import random

# 1. CREACIÓN DE LA TABLA CANAL DE VENTA

In [2]:
canal_venta = pd.DataFrame({
            'id_canal': [1, 2, 3, 4],
            'nombre_canal': ['kiosco', 'autoking', 'mostrador', 'comida de empleado']
        })
canal_venta

,id_canal,nombre_canal
0,1,kiosco
1,2,autoking
2,3,mostrador
3,4,comida de empleado


# 2. CREACIÓN DE LA TABLA TICKET

### ¿Cuántos tickets genero según el mes y año? Teniendo en cuenta los ingresos anuales de BK y la estacionalidad mensual de sus ventas

Estacionalidad anual según los ingresos anuales de Burguer King a nivel mundial

In [3]:
estacionalidad_anual_original={2018: 1, 2019: 1.0787, 2020:0.9696, 2021: 1.0969, 2022: 1.1454}


In [4]:
# Calcular la media original
media_original = sum(estacionalidad_anual_original.values()) / len(estacionalidad_anual_original)
media_original

1.05812

In [5]:
# Ajustar los índices para que la media sea 1
nuevo_valor_deseado = 1
estacionalidad_anual= {año: valor / media_original * nuevo_valor_deseado for año, valor in estacionalidad_anual_original.items()}
media_estacionalidad_anual = np.mean(list(estacionalidad_anual.values()))
media_estacionalidad_anual
print(f"Estacionalidad_anual ajustada: {estacionalidad_anual}")
print(f"Media de la estacionalidad_anual ajustada: {media_estacionalidad_anual}")

Estacionalidad_anual ajustada: {2018: 0.945072392545269, 2019: 1.0194495898385816, 2020: 0.9163421918118928, 2021: 1.0366499073829056, 2022: 1.0824859184213511}
Media de la estacionalidad_anual ajustada: 1.0


Estacionalidad mensual según los ingresos mensuales

In [6]:
estacionalidad_meses_original = {
    "enero": 1.17,
    "febrero": 1.06,
    "marzo": 1.01,
    "abril": 1.06,
    "mayo": 1.04,
    "junio": 0.88,
    "julio": 0.8,
    "agosto": 0.77,
    "septiembre": 0.91,
    "octubre": 1.0,
    "noviembre": 1.09,
    "diciembre": 1.2
}

In [7]:
# Calcular la media original
media_estacionalidad_meses_original = np.mean(list(estacionalidad_meses_original.values()))
media_estacionalidad_meses_original

0.9991666666666666

In [8]:
# Calcular la nueva estacionalidad_meses ajustada para tener una media de 1
nuevo_valor_deseado = 1
estacionalidad_meses = {mes: valor / media_estacionalidad_meses_original * nuevo_valor_deseado 
                                  for mes, valor in estacionalidad_meses_original.items()}

# Calcular la nueva media de estacionalidad_meses
media_nueva_estacionalidad_meses = np.mean(list(estacionalidad_meses.values()))

print(f"Estacionalidad_meses ajustada: {estacionalidad_meses}")
print(f"Media de la estacionalidad_meses ajustada: {media_nueva_estacionalidad_meses}")


Estacionalidad_meses ajustada: {'enero': 1.170975813177648, 'febrero': 1.0608840700583821, 'marzo': 1.010842368640534, 'abril': 1.0608840700583821, 'mayo': 1.0408673894912428, 'junio': 0.8807339449541285, 'julio': 0.8006672226855713, 'agosto': 0.7706422018348624, 'septiembre': 0.9107589658048374, 'octubre': 1.0008340283569641, 'noviembre': 1.090909090909091, 'diciembre': 1.201000834028357}
Media de la estacionalidad_meses ajustada: 1.0


### Creo 60 Data Frames para cada mes y año del periodo que queremos que abarque nuestro Data Frame Final

In [9]:
# Número de filas base
num_filas_base = 12000

# Contador global para el ID del ticket
id_ticket_actual = 1

# Diccionario para almacenar los DataFrames
tickets_mes_año = {}

# Crear los DataFrames
for año, est_anual in estacionalidad_anual.items():
    for mes, est_mensual in estacionalidad_meses.items():
        nombre_df = f"tickets_{mes}_{año}"
        num_filas = int(num_filas_base * est_anual * est_mensual)

        # Crear el DataFrame con el ID del ticket actualizado
        df = pd.DataFrame({
            'id_ticket': np.arange(id_ticket_actual, id_ticket_actual + num_filas),
            'id_canal': np.random.choice(canal_venta['id_canal'],
                                            p=[0.49, 0.26, 0.23, 0.02],
                                            size=num_filas)
        })

        # Obtener el número total de días en el mes y año especificados
        total_dias = calendar.monthrange(año, list(estacionalidad_meses.keys()).index(mes) + 1)[1]

        # Generar un valor aleatorio para el día
        df['dia'] = [random.randint(1, total_dias) for _ in range(num_filas)]

        # Construir la fecha con el día aleatorio
        df['fecha'] = pd.to_datetime(df.apply(lambda row: f"{año}-{list(estacionalidad_meses.keys()).index(mes) + 1:02d}-{row['dia']:02d}", axis=1))

        # Elimino la columna día
        df=df.drop('dia', axis=1)

        # Actualizar el contador global
        id_ticket_actual += num_filas

        # Almacenar el DataFrame en el diccionario
        tickets_mes_año[nombre_df] = df

        print(f"{nombre_df} creado con {num_filas} filas.")


tickets_enero_2018 creado con 13279 filas.
tickets_febrero_2018 creado con 12031 filas.
tickets_marzo_2018 creado con 11463 filas.
tickets_abril_2018 creado con 12031 filas.
tickets_mayo_2018 creado con 11804 filas.
tickets_junio_2018 creado con 9988 filas.
tickets_julio_2018 creado con 9080 filas.
tickets_agosto_2018 creado con 8739 filas.
tickets_septiembre_2018 creado con 10328 filas.
tickets_octubre_2018 creado con 11350 filas.
tickets_noviembre_2018 creado con 12371 filas.
tickets_diciembre_2018 creado con 13620 filas.
tickets_enero_2019 creado con 14325 filas.
tickets_febrero_2019 creado con 12978 filas.
tickets_marzo_2019 creado con 12366 filas.
tickets_abril_2019 creado con 12978 filas.
tickets_mayo_2019 creado con 12733 filas.
tickets_junio_2019 creado con 10774 filas.
tickets_julio_2019 creado con 9794 filas.
tickets_agosto_2019 creado con 9427 filas.
tickets_septiembre_2019 creado con 11141 filas.
tickets_octubre_2019 creado con 12243 filas.
tickets_noviembre_2019 creado con

Visualizo un dataframe creado

In [10]:
tickets_mes_año["tickets_diciembre_2022"]

,id_ticket,id_canal,fecha
0,704372,3,2022-12-23
1,704373,1,2022-12-12
2,704374,1,2022-12-12
3,704375,2,2022-12-04
4,704376,3,2022-12-08
...,...,...,...
15595,719967,1,2022-12-10
15596,719968,3,2022-12-08
15597,719969,1,2022-12-23
15598,719970,3,2022-12-09


Compruebo que se han aplicado bien las estacionalidades para generar el número de tickets.

In [11]:
# Inicializar una variable para acumular el recuento total de filas
total_filas = 0

# Calcular el recuento de número de filas para cada DataFrame
for nombre_df, df in tickets_mes_año.items():
    recuento_filas = len(df)  # Opcionalmente, puedes usar df.shape[0]
    total_filas += recuento_filas

# Calcular la media del número de filas
media_filas = total_filas / len(tickets_mes_año)
print(f"\nLa media del número de filas de todos los DataFrames es: {media_filas}")


La media del número de filas de todos los DataFrames es: 11999.516666666666


Concateno todos los tickets en un solo dataframe para poder tenerlos juntos y exportarlos a MySQL.

In [12]:
tickets_totales = pd.concat(tickets_mes_año.values(), ignore_index=True)

In [13]:
tickets_totales

,id_ticket,id_canal,fecha
0,1,3,2018-01-07
1,2,1,2018-01-22
2,3,1,2018-01-05
3,4,1,2018-01-19
4,5,3,2018-01-11
...,...,...,...
719966,719967,1,2022-12-10
719967,719968,3,2022-12-08
719968,719969,1,2022-12-23
719969,719970,3,2022-12-09


# 3. CREACIÓN DE LA TABLA CATEGORÍA PRODUCTO

In [72]:
categorias_lst= [
    'menus_parrilla',
    'menus_pollo',
    'menus_vegetarianos',
    'menus_singluten',
    'king_jr',
    'hamburguesas',
    'postres',
    'complementos_y_salsas',
    'bebidas',
    'ensaladas'
]


In [73]:
len(categorias_lst)

10

In [88]:
categoria_producto= pd.DataFrame({'id_categoria_producto':np.arange(1, len(categorias_lst)+1), 
                                  'nombre_categoria_producto': categorias_lst
                                  })
categoria_producto

,id_categoria_producto,nombre_categoria_producto
0,1,menus_parrilla
1,2,menus_pollo
2,3,menus_vegetarianos
3,4,menus_singluten
4,5,king_jr
5,6,hamburguesas
6,7,postres
7,8,complementos_y_salsas
8,9,bebidas
9,10,ensaladas


# 3. CREACIÓN DE LA TABLA PRECIOS

### Cargo los precios recogidos en un establecimiento de Burger King en Madrid

Durante todo el proyecto los precios e ingresos se van a tratar en valores brutos.

In [75]:
menus_parrilla = {"brutal bacon (2 carnes) menu grande": 10.95, 
            "brutal bacon (1 carnes) menu grande": 8.95, 
            "angus grill (2 carnes) menu grande": 12.95,
            "angus grill (1 carnes) menu grande": 10.45,
            "duo bacon cheddar (2 carnes) menu grande": 10.95,
            "duo bacon cheddar (1 carnes) menu grande": 8.95,
            "the king huevo (2 carnes) menu grande": 11.95,
            "the king huevo (1 carnes) menu grande": 9.95,
            "doble whopper menu grande": 9.95,
            "whopper menu grande": 7.95,
            "triple whopper menu grande": 11.95,
            "whopper jr. menu grande": 6.95,
            "doble steakhouse menu grande": 10.95,
            "steakhouse menu grande": 8.95,
            "big king xxl menu grande": 9.95,
            "big king menu grande": 7.95,
            "doble cheeseburguer menu grande": 7.55,
            "doble cheese bacon xxl menu grande": 9.95,
            "doble cheeseburguer bacon menu grande": 7.95,
            "doble cheeseburguer bbq menu grande": 7.55,
            "cheeseburguer menu grande": 6.95,
            "hamburguesa menu grande": 5.95
            }

In [76]:
menus_pollo={"brutal bacon doble crispy chicken menu grande": 9.95,
             "brutal bacon crispy menu grande": 8.75,
             "king chicken menu grande": 8.95,
             "doble cbk menu grande": 9.25,
             "cbk menu grande": 8.25,
             "duo bacon cheddar doble crispy chicken menu grande": 8.95,
             "long chicken menu grande": 7.95,
             "doble crispy chicken menu grande": 8.95,
             "crispy chicken menu grande": 7.55,
             "big king pollo menu grande": 7.95,
             "chicken wrap menu grande": 7.95,
             "chicken nuggets menu grande": 7.75
}

In [77]:
menus_vegetarianos={"whopper vegetal menu grande": 7.95,
                    "big king vegetal menu grande": 7.95,
                    "long vegetal menu grande": 7.95,
                    "9 nuggets vegetales menu grande": 7.55,
}

In [78]:
menus_singluten={"menu grande whopper sin gluten": 7.95,
                 "menu grande hamburguesa sin gluten": 5.95,
                 "menu grande cheeseburguer sin gluten": 6.95,
}

In [79]:
king_jr={"king jr burguer":4.20,
         "king jr nuggets x4":4.20,
         "king jr nuggets vegetales":4.20,
         "king jr sin gluten":4.20,
}

In [80]:
hamburguesas={"brutal bacon (2 carnes)": 8.95,
              "brutal bacon (1 carne)": 6.95,
              "angus grill (2 carnes)": 11.95,
              "angus grill (1 carne)": 9.45,
              "duo bacon cheddar (2 carnes)": 8.95,
              "duo bacon cheddar (1 carne)":6.95,
              "the king huevo (2 carnes)": 8.45,
              "the king huevo (1 carne)": 6.95,
              "doble whopper": 6.95,
              "whopper": 5.45,
              "triple whopper": 8.45,
              "doble steakhouse": 8.45,
              "steakhouse": 6.45,
              "big king xxl": 6.95,
              "big king": 5.45,
              "doble cheeseburguer": 4.95,
              "doble cheese bacon xxl": 6.95,
              "doble cheeseburguer bacon": 5.25,
              "doble cheeseburguer bbq":4.95,
              "cheeseburguer":1.5,
              "hamburguesa": 2.45,
              "brutal bacon doble crispy chicken": 7.95,
              "brutal bacon crispy": 6.95,
              "king chicken": 6.45,
              "doble cbk": 7.25,
              "cbk": 5.75,
              "duo bacon cheddar doble crispy chicken": 8.25,
              "long chicken": 5.45,
              "doble crispy chicken": 5.95,
              "crispy chicken": 4.95,
              "big king pollo": 5.45,
              "chicken wrap": 4.95,
              "chicken burguer": 1.20,
              "9 chicken nuggets": 3.95,
              "whopper vegetal":5.45,
              "big king vegetal": 5.45,
              "long vegetal": 5.45,
              "nuggets vegetales x9":3.95,
              "whopper sin gluten": 5.45,
              "hamburguesa sin gluten": 3.45,
              "cheeseburguer sin gluten": 3.95
}

In [81]:
postres={"king fusion lacasitos": 2.95,
         "king fusion milka choco galleta": 2.95,
         "king fusion oreo": 2.95,
         "king fusion milka oreo": 2.95,
         "king fusion milka original": 2.95,
         "king fusion kit kat": 2.95,
         "king fusion nutella": 2.95,
         "oreo shake": 3.25,
         "oreo shake chocolate": 3.25,
         "classic shake chocolate": 3.25,
         "classic shake fresa": 3.25,
         "classic shake vainilla": 3.25,
         "gofre lacasitos":3.25,
         "brownie caliente lacasitos":3.25,
         "brownie caliente": 2.95,
         "gofre caliente": 2.95,
         "gofre caliente nutella": 2.95,
         "danonino petidino fresa": 1,
         "actimel 0 fresa": 1,
         "piña": 1
}

In [82]:
complementos_y_salsas={
    "patatas clasicas grandes": 2.95,
    "patatas clasicas medianas": 2.75,
    "patatas clasicas gigantes": 3.45,
    "patatas supreme grandes": 2.95,
    "patatas supreme medianas": 2.75,
    "patatas supreme gigantes": 3.45,  
    "king fries (clasicas) queso bacon cebolla crispy": 3.95,
    "king fries (clasicas) queso bacon": 3.75,
    "king fries (supreme) queso bacon": 3.75,
    "chicken fries x6": 2.95,
    "chicken fries x9": 3.95,
    "chicken fries x24": 9.95,
    "9 chicken nuggets": 3.95,
    "6 chicken nuggets": 2.95,
    "20 chicken nuggets": 6.75,
    "alitas de pollo x6": 4.95,
    "alitas de pollo x3": 3.25,
    "9 chill cheese bites": 4.45,
    "6 chili cheese bites": 3.45,
    "king aros de cebolla (x10)": 2.95,
    "king aros de cebolla (x7)": 2.75,
    "king aros de cebolla (x13)": 3.45,
    "chicken fries x24": 9.95,
    "cubo: 12 x alitas de pollo": 9.95,
    "cubo: 8 x alitas de pollo 10 x nuggets": 9.95,
    "cubo: 6 x king aros de cebolla 4 x alitas de pollo 6 x alitas de pollo": 8.45,
    "cubo: 6 x chili cheese bites 4 x alitas de pollo 6 x nuggets": 8.45,
    "cubo patatas supreme": 4.95,
    "cubo patatas clasicas": 4.95,
    "cubo 24 x nuggets": 8.95,
    "cubo 24 x nuggets vegetales": 9.95,
    "nuggets vegetales x6": 2.95,
    "nuggets vegetales x9": 3.95,
    "nuggets vegetales x20": 6.75,
    "king aros de cebolla x13": 3.45,
    "king aros de cebolla x10": 2.95,
    "king aros de cebolla x7": 2.75,
    "salsa miel y mostaza": 0.5,
    "salsa barbacoa": 0.5,
    "salsa queso": 0.5,
    "salsa mayonesa":0.5,
    "salsa sour cream": 0.5
}

In [83]:
bebidas={"refresco mediano": 2.25,
         "refresco grande": 2.45,
         "refresco gigante": 2.75,
         "coca cola creations 3000": 2.50,
         "monster hamilton 800ml": 2.75,
         "monster energy 500ml": 2.75,
         "monster ultra white 500ml": 2.75,
         "monster mango loco 500ml":2.75,
         "bebida de frutas tropicales": 1,
         "juver 10 frutas + 10 vitaminas": 1,
         "agua mineral": 1.80,
         "cerveza sin alcohol": 2.25,
         "cerveza mediana": 2.25,
         "cerveza grande": 2.45,
         "trina zero": 2.25,
         "caprisun tropical": 1.75,
         "coca cola mediana": 2.25, 
         "coca cola grande": 2.45,
         "coca cola gigante": 2.75,
         "coca cola zero mediana": 2.25, 
         "coca cola zero grande": 2.45,
         "coca cola zero gigante": 2.75,
         "fanta de naranja zero mediana": 2.25, 
         "fanta de naranja zero grande": 2.45,
         "fanta de naranja zero gigante": 2.75,  
         "fanta de limon zero mediana": 2.25, 
         "fanta de limon zero grande": 2.45,
         "fanta de limon zero gigante": 2.75, 
         "nestea zero mediana": 2.25, 
         "nestea zero grande": 2.45,
         "nestea zero gigante": 2.75,   
         "aquarius zero mediana": 2.25, 
         "aquarius zero grande": 2.45,
         "aquarius zero gigante": 2.75,
         "trina": 2.75
}

In [84]:
ensaladas={
    "ensalada orginal": 3.75,
    "ensalada delight con pollo crujiente": 5.75
}

### Creo un daframe inicial con el listado de todos sus productos y sus respectivos precios

In [93]:
categorias = {
    'menus_parrilla': 'menus parrilla',
    'menus_pollo': 'menus pollo',
    'menus_vegetarianos': 'menus vegetarianos',
    'menus_singluten': 'menus singluten',
    'king_jr': 'king jr',
    'hamburguesas': 'hamburguesas',
    'postres': 'postres',
    'complementos_y_salsas': 'complementos y salsas',
    'bebidas': 'bebidas',
    'ensaladas': 'ensaladas'
}

# Inicializar una lista para almacenar los DataFrames individuales
dataframes_categorias = []

# Crear y concatenar los DataFrames
for nombre_dict, categoria in categorias.items():
    df_categoria = pd.DataFrame.from_dict(eval(nombre_dict), orient='index', columns=['precio']).reset_index()
    df_categoria['categoria_producto'] = nombre_dict
    df_categoria = df_categoria[['categoria_producto', 'index', 'precio']].rename(columns={'index': 'producto'})
    dataframes_categorias.append(df_categoria)

precios_df = pd.DataFrame()
precios_df = pd.concat(dataframes_categorias, ignore_index=True)
precios_df['id_producto'] = range(1, len(precios_df) + 1)
precios_df = precios_df[['id_producto'] + [col for col in precios_df.columns if col != 'id_producto']]
# Fusión de los DataFrames por el nombre de la categoría
precios_df = precios_df.merge(categoria_producto, left_on='categoria_producto', right_on='nombre_categoria_producto', how='left')
# Seleccionar las columnas relevantes
precios_df = precios_df[['id_producto', 'id_categoria_producto', 'producto', 'precio']]

precios_df

,id_producto,id_categoria_producto,producto,precio
0,1,1,brutal bacon (2 carnes) menu grande,10.95
1,2,1,brutal bacon (1 carnes) menu grande,8.95
2,3,1,angus grill (2 carnes) menu grande,12.95
3,4,1,angus grill (1 carnes) menu grande,10.45
4,5,1,duo bacon cheddar (2 carnes) menu grande,10.95
...,...,...,...,...
179,180,9,aquarius zero grande,2.45
180,181,9,aquarius zero gigante,2.75
181,182,9,trina,2.75
182,183,10,ensalada orginal,3.75


In [60]:
dataframes_categorias

[   categoria_producto                                  producto  precio
 0      menus parrilla       brutal bacon (2 carnes) menu grande   10.95
 1      menus parrilla       brutal bacon (1 carnes) menu grande    8.95
 2      menus parrilla        angus grill (2 carnes) menu grande   12.95
 3      menus parrilla        angus grill (1 carnes) menu grande   10.45
 4      menus parrilla  duo bacon cheddar (2 carnes) menu grande   10.95
 5      menus parrilla  duo bacon cheddar (1 carnes) menu grande    8.95
 6      menus parrilla     the king huevo (2 carnes) menu grande   11.95
 7      menus parrilla     the king huevo (1 carnes) menu grande    9.95
 8      menus parrilla                 doble whopper menu grande    9.95
 9      menus parrilla                       whopper menu grande    7.95
 10     menus parrilla                triple whopper menu grande   11.95
 11     menus parrilla                   whopper jr. menu grande    6.95
 12     menus parrilla              doble steakhous

In [25]:
precios_df = precios_df.rename(columns={'precio': 'precio_23'})
indices_por_año = {
    2022: {'precio_2S': 0.978222261779609, 'precio_1S': 1.00714391455726},
    2021: {'precio_2S': 0.944911424086684, 'precio_1S': 0.946065927485161},
    2020: {'precio_2S': 0.987597238744351, 'precio_1S': 1.00917096495306},
    2019: {'precio_2S': 0.966473937563194, 'precio_1S': 0.973633003841402},
    2018: {'precio_2S': 0.928686251688925, 'precio_1S': 0.947790247463161}
}

# Aplicar los cálculos usando un bucle
for año, indices in indices_por_año.items():
    for periodo, indice in indices.items():
        columna_destino = f'{periodo}_{año}'
        precios_df[columna_destino] = (precios_df['precio_23'] * indice).round(2)

# Mostrar el DataFrame resultante
precios_df

,id_producto,categoria_producto,producto,precio_23,precio_2S_2022,precio_1S_2022,precio_2S_2021,precio_1S_2021,precio_2S_2020,precio_1S_2020,precio_2S_2019,precio_1S_2019,precio_2S_2018,precio_1S_2018
0,1,menus parrilla,brutal bacon (2 carnes) menu grande,10.95,10.71,11.03,10.35,10.36,10.81,11.05,10.58,10.66,10.17,10.38
1,2,menus parrilla,brutal bacon (1 carnes) menu grande,8.95,8.76,9.01,8.46,8.47,8.84,9.03,8.65,8.71,8.31,8.48
2,3,menus parrilla,angus grill (2 carnes) menu grande,12.95,12.67,13.04,12.24,12.25,12.79,13.07,12.52,12.61,12.03,12.27
3,4,menus parrilla,angus grill (1 carnes) menu grande,10.45,10.22,10.52,9.87,9.89,10.32,10.55,10.10,10.17,9.70,9.90
4,5,menus parrilla,duo bacon cheddar (2 carnes) menu grande,10.95,10.71,11.03,10.35,10.36,10.81,11.05,10.58,10.66,10.17,10.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,180,bebidas,aquarius zero grande,2.45,2.40,2.47,2.32,2.32,2.42,2.47,2.37,2.39,2.28,2.32
180,181,bebidas,aquarius zero gigante,2.75,2.69,2.77,2.60,2.60,2.72,2.78,2.66,2.68,2.55,2.61
181,182,bebidas,trina,2.75,2.69,2.77,2.60,2.60,2.72,2.78,2.66,2.68,2.55,2.61
182,183,ensaladas,ensalada orginal,3.75,3.67,3.78,3.54,3.55,3.70,3.78,3.62,3.65,3.48,3.55


# 4. CREACIÓN DE LA TABLA PRODUCTOS/ ventas mysql

Comencé probando a hacer la tabla productos centrándome en unicamente un año y un mes.

In [26]:
# # Obtener el DataFrame para enero de 2018 de la lista
# tickets_enero_2018 = tickets_mes_año['tickets_enero_2018']

# # Calcular el número total de valores deseados
# num_valores_deseados = int(2.7 * len(tickets_enero_2018))

# # Crear la nueva tabla de productos con distribución de Poisson
# productos_enero_2018 = pd.DataFrame({
#     'id_ticket': np.repeat(tickets_enero_2018['id_ticket'], np.random.poisson(2.7, len(tickets_enero_2018)))
# })
# # Agregar la columna id_producto con valores random entre 1 y 184
# productos_enero_2018['id_producto'] = np.random.randint(1, 185, size=len(productos_enero_2018))

# # Merge con la tabla de precios para obtener la columna precio
# productos_enero_2018 = productos_enero_2018.merge(precios_df[['id_producto', 'precio_1S_2018']], on='id_producto', how='left')

# # Renombrar la columna de precio
# productos_enero_2018 = productos_enero_2018.rename(columns={'precio_1S_2018': 'precio'})

In [27]:
# # Obtener el DataFrame para enero de 2018 de la lista
# tickets_enero_2018 = tickets_mes_año['tickets_enero_2018']

# # Calcular el número total de valores deseados
# num_valores_deseados = int(2.7 * len(tickets_enero_2018))

# # Crear la nueva tabla de productos con distribución de Poisson
# productos_enero_2018 = pd.DataFrame({
#     'id_ticket': np.repeat(tickets_enero_2018['id_ticket'], np.random.poisson(2.7, len(tickets_enero_2018)))
# })

# # Agregar la columna 'id_producto' con valores random entre 1 y 184
# productos_enero_2018['id_producto'] = np.random.randint(1, 185, size=len(productos_enero_2018))

# # Merge con la tabla de precios para obtener la columna 'precio'
# productos_enero_2018 = productos_enero_2018.merge(precios_df[['id_producto', 'precio_1S_2018']], on='id_producto', how='left')

# # Renombrar la columna de 'precio'
# productos_enero_2018 = productos_enero_2018.rename(columns={'precio_1S_2018': 'precio'})

# # Añadir la columna 'unidades' con valores normalmente distribuidos
# media_unidades = 1.48
# desviacion_unidades = 0.97
# productos_enero_2018['unidades'] = np.clip(np.round(np.random.normal(media_unidades, desviacion_unidades, len(productos_enero_2018))), 1, 34).astype(int)

# # Agregar la columna 'ingresos' que es precio x unidades
# productos_enero_2018['ingresos']=productos_enero_2018['unidades']*productos_enero_2018['precio']

In [28]:
#productos_enero_2018.id_ticket.value_counts().mean()

In [29]:
#productos_enero_2018.id_ticket.value_counts().max()

In [30]:
#productos_enero_2018.id_ticket.value_counts().min()

In [31]:
#productos_enero_2018

In [32]:
#productos_enero_2018.describe(include='all').T

In [33]:
# # Calcular los ingresos totales por id_ticket
# ingresos_por_id_ticket = productos_enero_2018.groupby('id_ticket')['ingresos'].sum()

# # Calcular la media de los ingresos por id_ticket
# ingreso_medio_por_id_ticket = ingresos_por_id_ticket.mean()

# print(f"Ingreso medio por id_ticket: {ingreso_medio_por_id_ticket}")


Comienzo ya a crear el dataframe iterando entre meses y años

In [34]:
productos_mes_año = {}
month_to_number = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
    "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}
# Iteratar entre los años y meses
for año, est_anual in estacionalidad_anual.items():
    for mes, est_mensual in estacionalidad_meses.items():
        nombre_df=f"productos_{mes}_{año}"
        # Luego puedes construir el nombre de la clave
        key = f"tickets_{mes}_{año}"
        # Ahora puedes acceder al dataframe usando la clave corregida
        df = tickets_mes_año.get(key)
        
        # Calcula el número de valores deseados
        num_valores_deseados = int(2.7 * len(df))
        
        # Comienza a crear la columna id_ticket
        # Generar un número de repeticiones entero para cada id_ticket
        num_repeticiones = np.round(np.random.normal(2.7, 0.5, len(df)))

        # Ajustar el número de repeticiones para estar en el rango deseado (2 a 3.5)
        num_repeticiones = np.clip(num_repeticiones, 2, 3.5)

        # Crear el DataFrame productos con las repeticiones ajustadas
        productos = pd.DataFrame({
            'id_ticket': np.repeat(df['id_ticket'], num_repeticiones.astype(int))
        })

        # Añade la columna id_productos con valores random
        productos['id_producto'] = np.random.randint(1, 185, size=len(productos))

        mes_numero = month_to_number[mes]
        
        # Merge con la apropiada columna precio
        periodo = 1 if mes_numero <= 6 else 2
        precio_column = f"precio_{periodo}S_{año}"
        productos = productos.merge(precios_df[['id_producto', precio_column]], on='id_producto', how='left')
        productos = productos.rename(columns={precio_column: 'precio'})
        
        # Añade la columna 'unidades' con una distribución normal los valores
        media_unidades = 1.48
        desviacion_unidades = 0.97
        productos['unidades'] = np.clip(np.round(np.random.normal(media_unidades, desviacion_unidades, len(productos))), 1, 34).astype(int)
        
        # Añade la columna 'ingresos' multiplicando uds por su respectivo precio
        productos['ingresos'] = productos['unidades'] * productos['precio']

        # Fusionar los DataFrames 'productos' y 'tickets_mes_año' en base a la columna 'id_ticket'
        productos = productos.merge(tickets_mes_año[key][['id_ticket', 'fecha']], on='id_ticket', how='left')

        # Almacenar el DataFrame en la lista
        productos_mes_año[nombre_df]=productos


In [35]:
productos_mes_año.keys()

dict_keys(['productos_enero_2018', 'productos_febrero_2018', 'productos_marzo_2018', 'productos_abril_2018', 'productos_mayo_2018', 'productos_junio_2018', 'productos_julio_2018', 'productos_agosto_2018', 'productos_septiembre_2018', 'productos_octubre_2018', 'productos_noviembre_2018', 'productos_diciembre_2018', 'productos_enero_2019', 'productos_febrero_2019', 'productos_marzo_2019', 'productos_abril_2019', 'productos_mayo_2019', 'productos_junio_2019', 'productos_julio_2019', 'productos_agosto_2019', 'productos_septiembre_2019', 'productos_octubre_2019', 'productos_noviembre_2019', 'productos_diciembre_2019', 'productos_enero_2020', 'productos_febrero_2020', 'productos_marzo_2020', 'productos_abril_2020', 'productos_mayo_2020', 'productos_junio_2020', 'productos_julio_2020', 'productos_agosto_2020', 'productos_septiembre_2020', 'productos_octubre_2020', 'productos_noviembre_2020', 'productos_diciembre_2020', 'productos_enero_2021', 'productos_febrero_2021', 'productos_marzo_2021', 

In [36]:
productos_mes_año['productos_enero_2018']

,id_ticket,id_producto,precio,unidades,ingresos,fecha
0,1,140,3.27,1,3.27,2018-01-07
1,1,37,7.53,1,7.53,2018-01-07
2,1,140,3.27,1,3.27,2018-01-07
3,2,175,2.61,1,2.61,2018-01-22
4,2,159,2.13,2,4.26,2018-01-22
...,...,...,...,...,...,...
35270,13278,162,2.13,1,2.13,2018-01-24
35271,13278,79,3.74,1,3.74,2018-01-24
35272,13279,155,2.61,1,2.61,2018-01-05
35273,13279,51,6.59,2,13.18,2018-01-05


In [37]:

for df_key, df in productos_mes_año.items():
    # Verifica si la columna 'ingresos' está presente en el DataFrame
    ingresos=df['ingresos'].sum()
    print(f'ingresos= {ingresos}')

ingresos= 286748.27
ingresos= 259295.69
ingresos= 245422.32999999996
ingresos= 256321.90999999997
ingresos= 255020.58
ingresos= 215949.81000000003
ingresos= 193225.66
ingresos= 185449.65999999997
ingresos= 218767.65999999997
ingresos= 239743.27000000002
ingresos= 260614.72999999998
ingresos= 288761.83
ingresos= 316459.92
ingresos= 286821.5
ingresos= 273074.77999999997
ingresos= 286536.77
ingresos= 283549.55
ingresos= 239151.63999999998
ingresos= 214192.88
ingresos= 207170.40999999997
ingresos= 244405.56999999995
ingresos= 267475.88
ingresos= 291055.11
ingresos= 325968.62
ingresos= 295147.13999999996
ingresos= 265765.1
ingresos= 256521.23
ingresos= 266430.27
ingresos= 264813.48
ingresos= 223089.53
ingresos= 197894.41999999998
ingresos= 192150.51
ingresos= 225451.94
ingresos= 246412.52
ingresos= 270851.12999999995
ingresos= 298752.45
ingresos= 314176.6599999999
ingresos= 284960.64999999997
ingresos= 271700.77999999997
ingresos= 284731.99
ingresos= 276404.47
ingresos= 236020.68999999994
i

In [38]:
# Calcular estacionalidades esperadas
estacionalidades_esperadas = []

for año, est_anual in estacionalidad_anual.items():
    for mes, est_mensual in estacionalidad_meses.items():
        estacionalidades_esperadas.append(est_anual * est_mensual)

# Obtener ingresos reales del diccionario
ingresos_reales = []

for df_key, df in productos_mes_año.items():
    # Verificar si la columna 'ingresos' está presente en el DataFrame
    if 'ingresos' in df.columns:
        # Sumar los valores de la columna 'ingresos'
        ingreso_mensual = df['ingresos'].sum()
        ingresos_reales.append(ingreso_mensual)

# Comparar los ingresos reales con las estacionalidades esperadas
for i, ingreso_real in enumerate(ingresos_reales):
    mes, año = df_key.split('_')[1], df_key.split('_')[2]
    estacionalidad_esperada = estacionalidades_esperadas[i]
    
    print(f"Ingresos reales: {ingreso_real:.2f}")
    print(f"Estacionalidad esperada: {estacionalidad_esperada:.2f}")
    print("---")


Ingresos reales: 286748.27
Estacionalidad esperada: 1.11
---
Ingresos reales: 259295.69
Estacionalidad esperada: 1.00
---
Ingresos reales: 245422.33
Estacionalidad esperada: 0.96
---
Ingresos reales: 256321.91
Estacionalidad esperada: 1.00
---
Ingresos reales: 255020.58
Estacionalidad esperada: 0.98
---
Ingresos reales: 215949.81
Estacionalidad esperada: 0.83
---
Ingresos reales: 193225.66
Estacionalidad esperada: 0.76
---
Ingresos reales: 185449.66
Estacionalidad esperada: 0.73
---
Ingresos reales: 218767.66
Estacionalidad esperada: 0.86
---
Ingresos reales: 239743.27
Estacionalidad esperada: 0.95
---
Ingresos reales: 260614.73
Estacionalidad esperada: 1.03
---
Ingresos reales: 288761.83
Estacionalidad esperada: 1.14
---
Ingresos reales: 316459.92
Estacionalidad esperada: 1.19
---
Ingresos reales: 286821.50
Estacionalidad esperada: 1.08
---
Ingresos reales: 273074.78
Estacionalidad esperada: 1.03
---
Ingresos reales: 286536.77
Estacionalidad esperada: 1.08
---
Ingresos reales: 283549.

In [39]:
productos_totales=pd.concat(productos_mes_año.values(), ignore_index=True)

In [40]:
productos_totales

,id_ticket,id_producto,precio,unidades,ingresos,fecha
0,1,140,3.27,1,3.27,2018-01-07
1,1,37,7.53,1,7.53,2018-01-07
2,1,140,3.27,1,3.27,2018-01-07
3,2,175,2.61,1,2.61,2018-01-22
4,2,159,2.13,2,4.26,2018-01-22
...,...,...,...,...,...,...
1911473,719970,79,3.86,1,3.86,2022-12-09
1911474,719970,111,2.69,1,2.69,2022-12-09
1911475,719971,110,2.89,2,5.78,2022-12-17
1911476,719971,61,4.84,1,4.84,2022-12-17


In [41]:
productos_totales[productos_totales.id_ticket==12]

,id_ticket,id_producto,precio,unidades,ingresos,fecha
30,12,182,2.61,2,5.22,2018-01-11
31,12,157,0.95,1,0.95,2018-01-11


# 5. AÑADIMOS LAS VENTAS EN EUROS AL DATAFRAME TICKETS_TOTALES

In [42]:
# Agrupa por 'id_ticket' y suma los ingresos
ingresos_por_ticket = productos_totales.groupby('id_ticket')['ingresos'].sum().reset_index()

# Fusiona 'tickets_totales' y 'ingresos_por_ticket' en base a 'id_ticket'
tickets_totales = tickets_totales.merge(ingresos_por_ticket, on='id_ticket', how='left')

# Renombra la columna resultante
tickets_totales = tickets_totales.rename(columns={'ingresos': 'ventas_euros'})

# Muestra el DataFrame actualizado
tickets_totales

,id_ticket,id_canal,fecha,ventas_euros
0,1,3,2018-01-07,14.07
1,2,1,2018-01-22,14.70
2,3,1,2018-01-05,26.54
3,4,1,2018-01-19,26.16
4,5,3,2018-01-11,19.32
...,...,...,...,...
719966,719967,1,2022-12-10,16.42
719967,719968,3,2022-12-08,20.98
719968,719969,1,2022-12-23,15.36
719969,719970,3,2022-12-09,6.55


In [43]:
# import matplotlib.pyplot as plt

# # Crea un gráfico temporal
# plt.figure(figsize=(12, 6))
# plt.plot(tickets_totales['fecha'], tickets_totales['ventas_euros'], marker='o', linestyle='-')

# # Configura el eje x para mostrar etiquetas de mes y año
# plt.xticks(tickets_totales['fecha'], tickets_totales['fecha'].dt.strftime('%Y-%m'), rotation=45)

# # Etiquetas y título
# plt.xlabel('Fecha')
# plt.ylabel('Ventas en Euros')
# plt.title('Ventas Mensuales')

# # Muestra el gráfico
# plt.tight_layout()
# plt.show()

In [44]:
tickets_totales[tickets_totales['ventas_euros'].isna()==True]

,id_ticket,id_canal,fecha,ventas_euros
